In [1]:
import sys
sys.path.append("/Users/zhengwenjie/AI/work/ML_3/2017-CCF-BDCI-AIJudge")

In [4]:
# -*- encoding:utf-8 -*-
import codecs
import subprocess
from collections import namedtuple

import numpy as np
import pandas as pd
from gensim.models import Doc2Vec
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from utils import LOGGER
from config.db_config import Config
import warnings 
warnings.filterwarnings('ignore')
config = Config()

## 加载数据

In [5]:
data_df = pd.read_csv(config.data_csv_path, encoding='utf-8')
y = data_df['penalty'] - 1
rows = data_df.shape[0]

## 准备doc2vec 格式数据

In [6]:
SentimentDocument = namedtuple('SentimentDocument', 'words tags')
class Doc_list(object):

    def __init__(self, f):
        self.f = f
    def __iter__(self):
        for i,line in enumerate(codecs.open(self.f,encoding='utf8')):
            words = line.strip().split(' ')
            tags = [int(words[0][2:])]
            words = words[1:]
            yield SentimentDocument(words,tags) 
            
def write_d2v_data_path(content_list):
    doc_f = codecs.open(config.feat_d2v, encoding='utf-8', mode='w')
    for i, content in enumerate(content_list):
        if i % 10000 == 0:
            LOGGER.log('iter=%d' % i)
        doc_f.write(u'_*{} {}\n'.format(i, content))
    doc_f.close()

write_d2v_data_path(data_df['content'])

2019-03-28 22:00:15 iter=0


In [7]:
text="_*9 公诉 机关 平湖市 人民检察院 被告人 童某 个体 拆房 因涉嫌 盗窃罪 2012 平湖市 公安局 刑事拘留 同年 依法逮捕 羁押于 本市 看守所 平湖市 人民检察院 以平检 刑诉 2012 525 起诉书 指控 被告人 某犯 盗窃罪 2012 日向 本院 提起公诉 本院 同日 受理 依法 简易程序 实行 独任 审判 公开 开庭审理 本案 被告人 童某 到庭 参加 诉讼 现已 审理 终结 2008 时许 被告人 童某 童得明 判刑 本市 曹桥 街道 愚桥 村大 竹园 废品收购 被告人 童某 路边 望风 童得明 采用 搭线 手段 得失 停放 该处 华益牌 电瓶 三轮车 一辆 价值 2475 查明 赃物 电瓶 三轮车 扣押 发还 失主 上述事实 被告人 童某 开庭审理 过程 无异议 接受 刑事案件 登记表 失主 陈述 证人 证言 同伙 供述 刑事 判决书 辨认 笔录 照片 身份证明 搜查 笔录 扣押 发还 物品 清单 价格 鉴定结论 抓获 证据 予以 证实 足以认定"

In [8]:
words = text.strip().split(' ')
tags = [int(words[0][2:])]
words = words[1:]
print(words[0:4])
print(tags)
sd = SentimentDocument(words,tags)
sd

['公诉', '机关', '平湖市', '人民检察院']
[9]


SentimentDocument(words=['公诉', '机关', '平湖市', '人民检察院', '被告人', '童某', '个体', '拆房', '因涉嫌', '盗窃罪', '2012', '平湖市', '公安局', '刑事拘留', '同年', '依法逮捕', '羁押于', '本市', '看守所', '平湖市', '人民检察院', '以平检', '刑诉', '2012', '525', '起诉书', '指控', '被告人', '某犯', '盗窃罪', '2012', '日向', '本院', '提起公诉', '本院', '同日', '受理', '依法', '简易程序', '实行', '独任', '审判', '公开', '开庭审理', '本案', '被告人', '童某', '到庭', '参加', '诉讼', '现已', '审理', '终结', '2008', '时许', '被告人', '童某', '童得明', '判刑', '本市', '曹桥', '街道', '愚桥', '村大', '竹园', '废品收购', '被告人', '童某', '路边', '望风', '童得明', '采用', '搭线', '手段', '得失', '停放', '该处', '华益牌', '电瓶', '三轮车', '一辆', '价值', '2475', '查明', '赃物', '电瓶', '三轮车', '扣押', '发还', '失主', '上述事实', '被告人', '童某', '开庭审理', '过程', '无异议', '接受', '刑事案件', '登记表', '失主', '陈述', '证人', '证言', '同伙', '供述', '刑事', '判决书', '辨认', '笔录', '照片', '身份证明', '搜查', '笔录', '扣押', '发还', '物品', '清单', '价格', '鉴定结论', '抓获', '证据', '予以', '证实', '足以认定'], tags=[9])

In [9]:
doc_list = Doc_list(config.feat_d2v)

## distributed bag of words －doc2vec 模型训练和预测

In [10]:
print("distributed bag of words ..........")
d2v = Doc2Vec(dm=0, size=100, negative=5, hs=0, min_count=3, window=30, sample=1e-5, workers=8, alpha=0.025,min_alpha=0.025)
d2v.build_vocab(doc_list)

for i in range(5):
    LOGGER.log('pass:' + str(i))
    d2v.train(doc_list, total_examples=d2v.corpus_count, epochs=d2v.iter)
    X_d2v = np.array([d2v.docvecs[i] for i in range(rows)])
    '''
        Evaluate a score by cross-validation
        Array of scores of the estimator for each run of the cross validation

        scores：返回cv＝6 每次的得分
    '''
    scores = cross_val_score(LogisticRegression(C=3.0),X_d2v,y,cv=5) #ross_val_score(lasso, X, y)
    LOGGER.log('dbow: ' + str(scores) + ' ' + str(np.mean(scores)))

distributed bag of words ..........
2019-03-28 22:00:17 pass:0
2019-03-28 22:00:17 dbow: [0.30434783 0.31818182 0.23809524 0.33333333 0.5625    ] 0.3512916431394692
2019-03-28 22:00:17 pass:1
2019-03-28 22:00:18 dbow: [0.43478261 0.31818182 0.23809524 0.33333333 0.5       ] 0.36487859966120834
2019-03-28 22:00:18 pass:2
2019-03-28 22:00:18 dbow: [0.39130435 0.36363636 0.23809524 0.33333333 0.5       ] 0.36527385657820444
2019-03-28 22:00:18 pass:3
2019-03-28 22:00:18 dbow: [0.34782609 0.36363636 0.23809524 0.33333333 0.5       ] 0.35657820440429133
2019-03-28 22:00:18 pass:4
2019-03-28 22:00:18 dbow: [0.34782609 0.27272727 0.23809524 0.33333333 0.5       ] 0.33839638622247314


In [13]:
d2v.save(config.model_d2v_dbow)
LOGGER.log('Save done!')

2019-03-28 22:00:45 Save done!


In [18]:
# 获取编号“0” 文档的向量
d2v.docvecs[0]

array([-0.20439804, -0.04536807, -0.01064888, -0.27962515,  0.03723795,
       -0.07628016, -0.09852917, -0.23500569,  0.19499229, -0.20717788,
       -0.02695188, -0.16233839, -0.02680836, -0.06450634,  0.09936735,
        0.07089463,  0.1180456 , -0.03552295,  0.09178151, -0.33912387,
       -0.0420421 ,  0.14866246, -0.06491967, -0.14977627,  0.0825924 ,
        0.04756051,  0.02309666, -0.0400833 , -0.05145013, -0.08553205,
       -0.11951987,  0.03215332,  0.15663576,  0.03454518, -0.19814734,
        0.05819818, -0.17084774,  0.01311904, -0.24051586, -0.08189647,
       -0.05384031,  0.09810992,  0.01274253, -0.02541074,  0.16730721,
       -0.16204193, -0.17962602,  0.15194967, -0.05110624, -0.03862314,
       -0.18381326,  0.05639948,  0.01246327, -0.10676739, -0.19105582,
       -0.0766048 ,  0.02736868,  0.00748024,  0.10232171, -0.07977441,
       -0.16150486, -0.2095442 ,  0.03265984,  0.03971203,  0.01211743,
       -0.00999495, -0.28400028,  0.07465234,  0.15976109,  0.16

## distributed memory -doc2vec 模型训练和预测

In [14]:
d2v = Doc2Vec(dm=1, size=100, negative=5, hs=0, min_count=3, window=30, sample=1e-5, workers=8, alpha=0.025,
              min_alpha=0.025)
d2v.build_vocab(doc_list)
for i in range(5):
    LOGGER.log('pass:' + str(i))
    d2v.train(doc_list, total_examples=d2v.corpus_count, epochs=d2v.iter)
    X_d2v = np.array([d2v.docvecs[i] for i in range(rows)])
    scores = cross_val_score(LogisticRegression(C=3.0), X_d2v, y, cv=5)  # ross_val_score(lasso, X, y)
    LOGGER.log('dm: ' + str(scores) + ' ' + str(np.mean(scores)))

2019-03-28 22:00:51 pass:0
2019-03-28 22:00:51 dm: [0.34782609 0.31818182 0.23809524 0.44444444 0.5       ] 0.3697095175356045
2019-03-28 22:00:51 pass:1
2019-03-28 22:00:51 dm: [0.30434783 0.31818182 0.28571429 0.33333333 0.4375    ] 0.3358154526632787
2019-03-28 22:00:51 pass:2
2019-03-28 22:00:51 dm: [0.39130435 0.31818182 0.23809524 0.33333333 0.5       ] 0.3561829474872953
2019-03-28 22:00:51 pass:3
2019-03-28 22:00:52 dm: [0.39130435 0.31818182 0.23809524 0.33333333 0.4375    ] 0.3436829474872953
2019-03-28 22:00:52 pass:4
2019-03-28 22:00:52 dm: [0.39130435 0.31818182 0.23809524 0.33333333 0.4375    ] 0.3436829474872953


In [15]:
d2v.save(config.model_d2v_dm)
LOGGER.log('Save done!')

2019-03-28 22:01:02 Save done!
